In [91]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import sys,os,glob
from pandas import HDFStore
from threading import Thread
from datetime import  datetime,timedelta
width = 13
base_link = "http://amk030.imces.ru/meteodata/AMK_030_BIN/2019/09_2019/"

In [92]:
class obrabotka_danniy(object):
#-----------------------------------------------------------------------------------------------------------------------      

    @staticmethod
    def get_all_links_file(base_link=base_link):
        page = requests.get(base_link)
        soup = BeautifulSoup(page.content)
        result = []
        for link in soup.find_all('a', href=True):
            if (link['href'][-4]=='.' and link['href'][-1]=='B'):
                result.append(base_link + link['href'])
        return result

    @staticmethod
    def create_dir_for_downloading_files(self):
        path = os.path.abspath('') + '\\' + str(urls[0].split('/')[-2])
        try:
            os.path.isdir(path)
            os.mkdir(path)
        except FileExistsError as e:
            print(e)
            
    @staticmethod
    def download_files(urls=None):
        if urls==None:
            urls = ["http://amk030.imces.ru/meteodata/AMK_030_BIN/2019/10_2019/10010008.19B",
                     "http://amk030.imces.ru/meteodata/AMK_030_BIN/2019/10_2019/10010018.19B",
                       "http://amk030.imces.ru/meteodata/AMK_030_BIN/2019/06_2019/06010030.19R"]
        for i,j in enumerate(urls):
            handle = urllib.request.urlopen(urls[i])
            fname = os.path.basename(urls[i])
            with open(fname, "wb") as f_handler:
                while True:
                    chunk = handle.read()
                    if not chunk:
                        break
                    f_handler.write(chunk)
        
                    
    @staticmethod
    def get_file_names_in_dir():
        file_names = []
        file_name = os.path.abspath('')+'\\'
        for i in glob.glob("*.19B"):
            file_names.append(file_name + i)
        return file_names
    
    @staticmethod
    def function_obrabotka(default=0):
        data_frame_list = []
        file_names = os.path.abspath('')+'\\' + glob.glob("*.19B")[0]
        if default==1:
            file_names = get_file_names_in_dir()
        file_names = get_file_names_in_dir()
        with pd.HDFStore('data_base.h5',  mode='w') as store:
            for i,j in enumerate(file_names):
                with open(j, "rb") as file:
                    content = file.read()
                    data_dopolnitelniy = np.frombuffer(content[:17], dtype=np.dtype('i1'))
                    data_dopolnitelniy = np.frombuffer(data_dopolnitelniy[:16], dtype=np.dtype('i2'))
                    date_temp = data_dopolnitelniy[:-1]
                    date_time = datetime(*date_temp).strftime("%d.%m.%Y %H:%M:%S.%f")
                    numpy_data_polojitelnie = np.frombuffer(content, dtype=np.dtype('B'))[17:-14]
                    numpy_data_polojitelnie = numpy_data_polojitelnie.reshape(int(len(numpy_data_polojitelnie) / width), width)
                    numpy_data_polojitelnie = np.delete(numpy_data_polojitelnie,12,1)
                    numpy_data_polojitelnie = np.frombuffer(numpy_data_polojitelnie, dtype=np.dtype('i2'))
                    numpy_data_polojitelnie = numpy_data_polojitelnie.reshape(int(len(numpy_data_polojitelnie)/6),6)
                    row = numpy_data_polojitelnie.shape[0]
                    column = numpy_data_polojitelnie.shape[1] + 1
                    zero = np.zeros((row,1))
                    numpy_data_polojitelnie = np.append(numpy_data_polojitelnie,zero,axis=1) 
                    for i in range(len(numpy_data_polojitelnie)):
                        numpy_data_polojitelnie[i,0] = numpy_data_polojitelnie[i,0] / 100
                        numpy_data_polojitelnie[i,1] = numpy_data_polojitelnie[i,1] / 100
                        numpy_data_polojitelnie[i,2] = numpy_data_polojitelnie[i,2] / 100
                        numpy_data_polojitelnie[i,3] = numpy_data_polojitelnie[i,3] / 100
                        numpy_data_polojitelnie[i,4] = numpy_data_polojitelnie[i,4] / 10
                        numpy_data_polojitelnie[i,5] = numpy_data_polojitelnie[i,5] / 100
                    datelist = pd.date_range(date_time, periods=len(numpy_data_polojitelnie), freq='12.5ms').to_pydatetime()
                    data_frame = pd.DataFrame({'Time': datelist, 'Температура': numpy_data_polojitelnie[:,0], 'Южный компонент': numpy_data_polojitelnie[:,1],
                                  'Восточный компонент': numpy_data_polojitelnie[:,2], 'Вертикальный компонент': numpy_data_polojitelnie[:,3],
                                  'Атмосферное давление': numpy_data_polojitelnie[:,4], 'Влажность воздуха': numpy_data_polojitelnie[:,5],
                                  'Признак ошибки': numpy_data_polojitelnie[:,6]})
                    data_frame_list.append(data_frame)
                
                    store.append('df', data_frame, data_columns= data_frame.columns, format='table')
                
        return data_frame_list
    
    
#############################################dopolnitelnie functions  
#-----------------------------------------------------BEGIN---------------------------------------------------------      
@staticmethod
def get_cut_links_files_for_threading(self): 
    links_files = get_all_links_file()
    cut_links_files = [[links_files[:1000]],
                       [links_files[1000:2000]],
                       [links_files[2000:3000]],
                       [links_files[3000:]]]
    return cut_links_files
@staticmethod    
def download_files_with_threading(self):
    func = obrabotka_danniy.download_files
    for i,j in enumerate(cut_links_files):
        my_thread = Thread(target=func, name=[i], args=(j,))
        my_thread.start()
        if i ==10:
            break
    
    
#------------------------------------------------------END-----------------------------------------------------------      


In [93]:
obrabotka_danniy.download_files()

In [99]:
asd  = obrabotka_danniy.function_obrabotka()
dataframe_database = pd.read_hdf('data_base.h5')
print(dataframe_database.mean())


Температура                 2.922400
Южный компонент            -0.396391
Восточный компонент         0.762942
Вертикальный компонент      0.064141
Атмосферное давление      737.550011
Влажность воздуха          83.137376
Признак ошибки              0.000000
dtype: float64


,Time,Температура,Южный компонент,Восточный компонент,Вертикальный компонент,Атмосферное давление,Влажность воздуха,Признак ошибки
0,2019-01-10 00:08:14.000343,2.33,-1.84,-1.26,1.15,743.7,93.43,0.0
1,2019-01-10 00:08:14.012843,2.42,-1.88,-1.22,1.02,743.8,93.44,0.0
2,2019-01-10 00:08:14.025343,2.32,-1.78,-1.03,0.81,743.7,93.43,0.0
3,2019-01-10 00:08:14.037843,2.37,-1.72,-1.05,0.71,743.7,93.44,0.0
4,2019-01-10 00:08:14.050343,2.37,-1.72,-1.01,0.58,743.7,93.44,0.0
5,2019-01-10 00:08:14.062843,2.37,-1.64,-1.11,0.67,743.7,93.44,0.0
6,2019-01-10 00:08:14.075343,2.34,-1.68,-1.16,0.67,743.7,93.43,0.0
7,2019-01-10 00:08:14.087843,2.37,-1.75,-1.15,0.60,743.7,93.43,0.0
8,2019-01-10 00:08:14.100343,2.40,-1.71,-1.20,0.65,743.7,93.42,0.0
9,2019-01-10 00:08:14.112843,2.40,-1.79,-1.19,0.69,743.7,93.42,0.0
